# Домашнее задание 2 
## Contrastive and non-contrastive methods in CXR images

## Оценивание

Задание должно быть выполнено самостоятельно. Похожие решения будут считаться плагиатом. Если вы опирались на внешний источник в реализации, необходимо указать ссылку на него. 

В качестве решения, необходимо предоставить код (`train.py` с аргументами для выбора датасета/метода) + отчет, в котором будут отображены все детали выбора гиперпараметров, комментарии, сопровождающие графики, а так же ответы на вопросы в ДЗ. Оформляйте отчет четко и читаемо. Плохо оформленный код, плохо оформленные графики негативно скажутся на оценке, так же как и неэффективная реализация.

## Введение

Вам предстоит реализовать (задание 0) и поработать с двумя методами - [SimCLR](https://arxiv.org/abs/2002.05709) и [VICReg](https://arxiv.org/abs/2105.04906). Обучать их будем на датасете, относящемся к домену медицинских изображений (задания 1-4). Подключим онлайн пробинг (задание 3), а так же сравним с трансфером с imagenet домена в этот мед домен (задание 4).

### Датасеты [MedMNIST+](https://medmnist.com/)

Будем использовать уже подготовленные сконвертированные из DICOM'ов картинки. MedMNIST включает в себя два релевантных для нас датасета с рентгеновскими снимками грудной клетки:


| MedMNIST2D     | Data Modality | Tasks (# Classes/Labels)           | # Samples | # Training | # Validation | # Test |
|----------------|---------------|------------------------------------|-----------|------------|--------------|--------|
| ChestMNIST     | Chest X-Ray   | Multi-Label (14), Binary-Class (2) | 112,120   | 78,468     | 11,219       | 22,433 |
| PneumoniaMNIST | Chest X-Ray   | Binary-Class (2)                   | 5,856     | 4,708      | 524          | 624    |

На этот раз будем использовать разрешение 224x224 (необходимо выставить `size` при инициализации датасета). Несколько картинок из ChestMNIST:

![CXR image examples from ChestMNIST](data/cxr.png)

## Задание 0 (2 балла)

Реализуйте SimCLR и VICReg на базе ResNet-18 энкодера. Для этого надо реализовать соответствующие лосс-функции и архитектуры проекционных голов. Убедитесь, в корректности реализации на CIFAR-10 (не забудьте применить коррекцию резнета для картинок разрешением 32x32 из предыдущего домашнего задания). Для этого, сначала сделайте предобучение на train части датасета в течении 100 эпох, затем сделайте линейный пробинг с замороженным выучившимся энкодером.

Референсный интервал top-1 accuracy для 100 эпох предобучения ~80-83% на линейном пробинге (если не получается, проверьте реализацию оптимизатора (**LARS**) и расписания шага обучения (`warmup_cosine`) или попробуйте подвигать learning rate).

**NB**
Чтобы сэкономить на психотерапевте, используйте оптимизатор [LARS](https://arxiv.org/abs/1708.03888) и `LinearWarmupCosineAnnealing` шедулер. Их нет в торче, но довольно просто реализовать самому или взять референсную реализацию.

In [1]:
from src.train import *

In [2]:
base_dataset = datasets.CIFAR10(root='./data', train=True, download=True)
ssl_dataset = SSLDataset(base_dataset, get_ssl_transforms())
ssl_loader = DataLoader(ssl_dataset, batch_size=256, shuffle=True, num_workers=4)

model = SimCLR()
model = pretrain_ssl(model, ssl_loader, epochs=100)

train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2470, 0.2435, 0.2616))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, transform=train_transform)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4)

linear_probe(model.encoder, train_loader, test_loader, epochs=100)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 170M/170M [00:46<00:00, 3.67MB/s]


Epoch 1/100, Loss: 5.3763
Epoch 2/100, Loss: 5.1443
Epoch 3/100, Loss: 5.1092
Epoch 4/100, Loss: 5.0813
Epoch 5/100, Loss: 5.0561
Epoch 6/100, Loss: 5.0277
Epoch 7/100, Loss: 4.9996
Epoch 8/100, Loss: 4.9709
Epoch 9/100, Loss: 4.9426
Epoch 10/100, Loss: 4.9165
Epoch 11/100, Loss: 4.8947
Epoch 12/100, Loss: 4.8723
Epoch 13/100, Loss: 4.8490
Epoch 14/100, Loss: 4.8310
Epoch 15/100, Loss: 4.8129
Epoch 16/100, Loss: 4.7960
Epoch 17/100, Loss: 4.7817
Epoch 18/100, Loss: 4.7700
Epoch 19/100, Loss: 4.7592
Epoch 20/100, Loss: 4.7475
Epoch 21/100, Loss: 4.7393
Epoch 22/100, Loss: 4.7301
Epoch 23/100, Loss: 4.7213
Epoch 24/100, Loss: 4.7156
Epoch 25/100, Loss: 4.7105
Epoch 26/100, Loss: 4.7060
Epoch 27/100, Loss: 4.6992
Epoch 28/100, Loss: 4.6948
Epoch 29/100, Loss: 4.6907
Epoch 30/100, Loss: 4.6859
Epoch 31/100, Loss: 4.6828
Epoch 32/100, Loss: 4.6757
Epoch 33/100, Loss: 4.6735
Epoch 34/100, Loss: 4.6689
Epoch 35/100, Loss: 4.6627
Epoch 36/100, Loss: 4.6628
Epoch 37/100, Loss: 4.6598
Epoch 38/1

## Задание 1 (1 балл)

Загрузите упомянутые датасеты из `MedMNIST+` и проанализируйте данные. Например, посмотрите на количество и баланс классов, как устроена разметка по классам, найдите среднее и дисперсию значений пикселей. Определите **подходящие метрики и лосс** для конечной задачи для **каждого** из датасетов, аргументировано объясните ваш выбор.

Это задание выполнено в "src/EDA.ipynb"

## Задание 2 (2 балла)

CXR изображения выглядят специфично. Кажется, что нужно иметь и специфичные для таких картинок аугментации.
Поиграйтесь с трансформами и зафиксируйте набор, с которым вы будете проводить финальные запуски предобучения.

### Каким образом можно определить подходящие аугментации?

За неимением экспертного знания (если есть знакомый врач-рентгенолог, можно посоветоваться), будем отталкиваться от набора аугментаций в естественных картинках. Начнем с набора, используемого в SimCLR-подобных методах, для ImageNet. Примерно так готовый набор выглядит в `torchvision`'е (обратите внимание, что при создании `СolorJitter` указываются не сами интвервалы, а дельта, т.е. `brightness=0.4` дает `(0.6, 1.4)`):

```python
Compose(
      RandomResizedCrop(
          size=(224, 224),
          scale=(0.08, 1.0),
          ratio=(0.75, 1.3333333333333333),
          interpolation=InterpolationMode.BICUBIC,
          antialias=True)
      RandomApply(
          ColorJitter(
              brightness=(0.6, 1.4),
              contrast=(0.6, 1.4),
              saturation=(0.8, 1.2),
              hue=(-0.1, 0.1)))
      RandomGrayscale(p=0.2)
      GaussianBlur(p=0.5)
      Solarization(p=0.1)
      RandomHorizontalFlip(p=0.5)
      ToTensor()
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.228, 0.224, 0.225], inplace=False)
```

Какие параметры аугментаций стоило бы поменять? Реализуйте набор трансформов, посмотрите какие картинки получаются на выходе (не забудьте перевести выход в нужный интервал значений для визуализации), поиграйтесь со значениями параметров (например, `scale` в `RandomResizedCrop` или `brightness` в `ColorJitter`). Какие трансформы стоит убрать? Попробуйте добавить инвертирование и повороты картинок на небольшой угол.

Рентгеновский снимок - это не отражение света, а проекция плотности тканей, поэтому дефолтный набор аугментаций будет работать плохо. 

Остальной анализ и подбор аугментаций описан в src/EDA.ipynb

Для chestmnist у нас явно задача несбаллансированной классификации в обоих случаях. В качестве лосса просто возьмем BCE и навесим на меньший класс class weight множитель. В качестве метрики в обоих случаях возьмем PR-AUC. Также можно будет дополнительно померять F-beta score для какого-то порога, так как мы работаем с медицинскими данными, хотелось бы значимость рекола выкуртить и брать F2/F3 score.

## Задание 3 (4 балла)

Для того, чтобы честно найти подходящий набор аугментаций, надо проводить этап предобучения и затем оценивать качество получившихся репрезентаций на конечной задаче. Можно ли это как-то ускорить? Раз у нас есть разметка для всего датасета, воспользуемся ей и ускорим подбор аугментаций с помощью online probing'a.
Для этого, добавим голову для линейного пробинга `linear_probe` к нашему энкодеру (`backbone`) и проекционной голове (`projection_head`). Эта линейная "проба" будет состоять из одного линейного слоя из размерности выхода энкодера (например, 512 для ResNet18) в число классов на конечной задаче (например, 14 классов у ChestMNIST).

На каждой итерации **предобучения** будем учить `backbone` и `projection_head` на претекстовую задачу (например, SimCLR лосс), а линейную пробу на классификацию.
Получается такая двуглавая архитектура, где градиенты с претекстового лосса текут по проекционной голове и энкодеру, а градиенты с классификационного лосса только по линейной пробе (не забудьте правильно `detach'`нуться).

```
                      projection_head(h) -> ssl_loss
                    /
x -> encoder(x) -> h
                    \
                      probe(h.detach()) -> cls_loss
```

Записать это можно примерно так:
```python
for batch in aug_dataloader:
  x1, x2, y = batch
  h1, h2 = model.backbone(x1, x2)
  z1, z2 = model.projection_head(h1, h2)
  logits = model.linear_probe(h1.detach())
  total_loss = ssl_loss(z1, z2) + cls_loss(yhat, y)
  total_loss.backward()
```

Таким образом, мы сможем в реальном времени наблюдать за тем, как обучение на претекстовую задачу влияет на качество репрезентаций для конечной задачи. Конечно, это не то же самое, что провести полный цикл предобучения, а затем измерить качество на конечной задаче. Тем не менее, это обеспечивает быструю итерацию по конфигурациям гиперпараметров (например, выбор аугментаций). Можно делать запуски на небольшое число эпох и сравнивать онлайн метрики.

*NB* Если вспомнить STL-10 из ДЗ 1, разметка была доступна только для небольшого подмножества (`train` vs `unlabeled`). В таком случае онлайн пробинг все еще можно делать, пробу можно обучать во время валидационной эпохи на размеченном сплите (веса энкодера заморожены).


### Этап отбора аугментаций

Добавьте онлайн пробинг в пайплайн обучения SimCLR. Воспользуемся результатами онлайн пробинга на ранних эпохах для отбора аугментаций. Предложите свой набор аугментаций исходя из общих соображений и анализа из **задания 2**, так же можно попробовать [RandAugment](https://docs.pytorch.org/vision/main/generated/torchvision.transforms.RandAugment.html). В качестве референсного набора зафиксируем следующую композицию трансформов:

```python
Compose(
      RandomApply(    
          RandomRotation(degrees=[-10.0, 10.0],
          interpolation=InterpolationMode.NEAREST,
          expand=False,
          fill=0))
      RandomResizedCrop(
          size=(224, 224),
          scale=(0.5, 1.0),
          ratio=(0.75, 1.3333333333333333),
          interpolation=InterpolationMode.BICUBIC, antialias=True)
      RandomApply(    
          ColorJitter(brightness=(0.8, 1.2), contrast=(0.8, 1.2)))
      RandomHorizontalFlip(p=0.5)
      RandomApply(    
          Lambda(<lambda>, types=['object']))
      ToTensor()
      Normalize(mean=[0.5], std=[0.5], inplace=False)
),
```
где `<lambda>` это функция для инвертирования изображения.
Сравните выбранный вами набор аугментаций и референсный, какой из них лучше? Для сравнения можете ориентироваться на метрики онлайн пробинга на 5-10 эпохах.

### Этап полного предобучения

Зафиксируйте "лучший" набор и выполните полное предобучение (например, 50 эпох) с методами реализованными **задания 0**: SimCLR и VICReg (не забудьте использовать версию ResNet18 для разрешения 224х224).
После предобучения проведите (офлайн) линейный пробинг на всех датасетах (ChestMNIST и PneumoniaMNIST). В отчете продемонстрируйте графики обучения (значение лосса, значение метрик онлайн пробинга в ходе обучения), а также таблицу с финальными результатами. Проанализируйте разницу между SimCLR и VICReg.


Итого, краткий план задания:
1. Сформируйте собственный набор аугментаций для CXR и добавьте референсный.

2. Для каждого набора: предобучение SimCLR 5–10 эпох с онлайн-пробингом; сравниваем метрики.

3. Выбираем лучший набор → полное предобучение: SimCLR — 20+ эпох, VICReg — 20+ эпох.

4. Выполняем офлайн-линейный пробинг и сравниваем SimCLR и VICReg.

**Бонусный балл** получат решения, у которых значения финальных метрик соответсвуют supervised качеству (т.е. как если бы вы обучали ResNet-18 с нуля на каждом датасете). Значения метрик при supervised обучении можно найти [здесь](https://medmnist.com/). 

#### 1. Сформируйте собственный набор аугментаций для CXR и добавьте референсный.

Выполнено в dataset.py 

#### 2. Для каждого набора: предобучение SimCLR 5–10 эпох с онлайн-пробингом; сравниваем метрики.

In [5]:
# In[1]:
%load_ext autoreload
%autoreload 2

import torch
from torch.utils.data import DataLoader
from src.dataset import CustomNPZDataset, SSLDataset, get_medmnist_transforms, HFDataset
from src.model import SimCLR, VICReg
from src.train import pretrain_ssl_with_online_probe, offline_linear_probe # pretrain_ssl

DATA_PATH = './data/pneumoniamnist_224.npz'
BATCH_SIZE = 256
NUM_WORKERS = 12
NUM_CLASSES = 2 # 2 for pneumonia, 14 for ChestMNIST
DEVICE = 'cuda'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
train_base_ds = CustomNPZDataset(DATA_PATH, split='train')
val_base_ds = CustomNPZDataset(DATA_PATH, split='val')

def run_augmentation_experiment(aug_type, epochs=5):
    print(f"\n>>> Running Online Probing with Augmentation: {aug_type}")
    ssl_transform = get_medmnist_transforms(size=224, augment=aug_type)
    val_transform = get_medmnist_transforms(size=224, augment=None)

    train_ssl_ds = SSLDataset(train_base_ds, ssl_transform)
    # train_ssl_ds = HFDataset(train_ssl_ds, for_ssl=True)

    # Reinstantiate val to attach transform
    val_ds_eval = CustomNPZDataset(DATA_PATH, split='val', transform=val_transform)
    # val_ds_eval = HFDataset(val_ds_eval, for_ssl=False)

    train_loader = DataLoader(train_ssl_ds, batch_size=BATCH_SIZE, shuffle=True,
                              num_workers=NUM_WORKERS, pin_memory=True)
    val_loader = DataLoader(val_ds_eval, batch_size=BATCH_SIZE, shuffle=False,
                           num_workers=NUM_WORKERS, pin_memory=True)

    model = SimCLR(encoder_dim=512, projection_dim=128).to(DEVICE)

    pretrain_ssl_with_online_probe(
        model,
        train_loader,
        val_loader,
        num_classes=NUM_CLASSES,
        epochs=epochs,
        lr=0.3,
        device=DEVICE,
        log_dir=f'runs/pneumonia_online_probe_{aug_type}'
    )


In [7]:
run_augmentation_experiment(aug_type="ref", epochs=10)


>>> Running Online Probing with Augmentation: ref
Class weights: [1.9390445 0.6737264]
Epoch 1/10 | SSL Loss: 5.8492 | Probe Loss: 0.4730 | Train Acc: 74.81% | LR: 0.000000
Epoch 2/10 | SSL Loss: 5.6237 | Probe Loss: 0.3288 | Train Acc: 85.28% | LR: 0.030000
Epoch 3/10 | SSL Loss: 5.5542 | Probe Loss: 0.3152 | Train Acc: 86.36% | LR: 0.060000
Epoch 4/10 | SSL Loss: 5.4496 | Probe Loss: 0.3169 | Train Acc: 85.68% | LR: 0.090000
Epoch 5/10 | SSL Loss: 5.3958 | Probe Loss: 0.3150 | Train Acc: 86.09% | LR: 0.120000
  -> Val ROC-AUC: 92.12% | Val Acc: 68.89%
Epoch 6/10 | SSL Loss: 5.4129 | Probe Loss: 0.3178 | Train Acc: 86.75% | LR: 0.150000
Epoch 7/10 | SSL Loss: 5.3266 | Probe Loss: 0.3028 | Train Acc: 86.43% | LR: 0.180000
Epoch 8/10 | SSL Loss: 5.2712 | Probe Loss: 0.3009 | Train Acc: 86.89% | LR: 0.210000
Epoch 9/10 | SSL Loss: 5.2112 | Probe Loss: 0.3153 | Train Acc: 86.53% | LR: 0.240000
Epoch 10/10 | SSL Loss: 5.1521 | Probe Loss: 0.3375 | Train Acc: 85.32% | LR: 0.270000
  -> Val

In [8]:
run_augmentation_experiment(aug_type="ssl", epochs=10)


>>> Running Online Probing with Augmentation: ssl
Class weights: [1.9390445 0.6737264]
Epoch 1/10 | SSL Loss: 5.7465 | Probe Loss: 0.6941 | Train Acc: 56.88% | LR: 0.000000
Epoch 2/10 | SSL Loss: 5.2766 | Probe Loss: 0.5527 | Train Acc: 71.26% | LR: 0.030000
Epoch 3/10 | SSL Loss: 5.1812 | Probe Loss: 0.4444 | Train Acc: 80.95% | LR: 0.060000
Epoch 4/10 | SSL Loss: 5.0078 | Probe Loss: 0.3727 | Train Acc: 82.73% | LR: 0.090000
Epoch 5/10 | SSL Loss: 4.9122 | Probe Loss: 0.3275 | Train Acc: 85.94% | LR: 0.120000
  -> Val ROC-AUC: 94.02% | Val Acc: 73.28%
Epoch 6/10 | SSL Loss: 4.8235 | Probe Loss: 0.3100 | Train Acc: 86.43% | LR: 0.150000
Epoch 7/10 | SSL Loss: 4.7778 | Probe Loss: 0.2954 | Train Acc: 86.70% | LR: 0.180000
Epoch 8/10 | SSL Loss: 4.7081 | Probe Loss: 0.2819 | Train Acc: 87.55% | LR: 0.210000
Epoch 9/10 | SSL Loss: 4.6384 | Probe Loss: 0.2877 | Train Acc: 87.28% | LR: 0.240000
Epoch 10/10 | SSL Loss: 4.5913 | Probe Loss: 0.2983 | Train Acc: 87.72% | LR: 0.270000
  -> Val

In [9]:
run_augmentation_experiment(aug_type="sft", epochs=10)


>>> Running Online Probing with Augmentation: sft
Class weights: [1.9390445 0.6737264]
Epoch 1/10 | SSL Loss: 5.7406 | Probe Loss: 0.7059 | Train Acc: 59.56% | LR: 0.000000
Epoch 2/10 | SSL Loss: 5.5298 | Probe Loss: 0.6000 | Train Acc: 63.64% | LR: 0.030000
Epoch 3/10 | SSL Loss: 5.4391 | Probe Loss: 0.5020 | Train Acc: 81.07% | LR: 0.060000
Epoch 4/10 | SSL Loss: 5.1513 | Probe Loss: 0.4122 | Train Acc: 81.88% | LR: 0.090000
Epoch 5/10 | SSL Loss: 4.9592 | Probe Loss: 0.3677 | Train Acc: 84.09% | LR: 0.120000
  -> Val ROC-AUC: 92.67% | Val Acc: 77.86%
Epoch 6/10 | SSL Loss: 4.9260 | Probe Loss: 0.3599 | Train Acc: 84.88% | LR: 0.150000
Epoch 7/10 | SSL Loss: 4.9370 | Probe Loss: 0.3352 | Train Acc: 85.71% | LR: 0.180000
Epoch 8/10 | SSL Loss: 4.7628 | Probe Loss: 0.3252 | Train Acc: 85.56% | LR: 0.210000
Epoch 9/10 | SSL Loss: 4.6533 | Probe Loss: 0.3181 | Train Acc: 86.32% | LR: 0.240000
Epoch 10/10 | SSL Loss: 4.6005 | Probe Loss: 0.3177 | Train Acc: 85.90% | LR: 0.270000
  -> Val

Ура, получилось победить дефолтные аугметнации, но только на несбаллансированной метрике, чего должно быть достаточно.  Интересно, что модель переобучается и дает хуже качество на 10 эпохах, возможно стоит сделать аугменнтации более агрессивными.

#### 3. Выбираем лучший набор → полное предобучение: SimCLR — 20+ эпох, VICReg — 20+ эпох.
#### 4. Выполняем офлайн-линейный пробинг и сравниваем SimCLR и VICReg.

In [6]:
import torch.nn as nn
from src.model import VICReg
# from src.train import pretrain_ssl, offline_linear_probe
from src.train import offline_linear_probe

BEST_AUG = "ssl"
PRETRAIN_EPOCHS = 50
EVAL_EPOCHS = 30

ssl_transform = get_medmnist_transforms(size=224, augment=BEST_AUG)
train_base_ds = CustomNPZDataset(DATA_PATH, split='train')
ssl_train_ds = SSLDataset(train_base_ds, ssl_transform)

ssl_loader = DataLoader(ssl_train_ds, batch_size=BATCH_SIZE, shuffle=True,
                       num_workers=NUM_WORKERS, pin_memory=True)

clean_transform = get_medmnist_transforms(size=224, augment=None)
train_eval_ds = CustomNPZDataset(DATA_PATH, split='train', transform=clean_transform)
test_eval_ds = CustomNPZDataset(DATA_PATH, split='test', transform=clean_transform)

train_eval_loader = DataLoader(train_eval_ds, batch_size=BATCH_SIZE, shuffle=True,
                               num_workers=NUM_WORKERS)
test_eval_loader = DataLoader(test_eval_ds, batch_size=BATCH_SIZE, shuffle=False,
                              num_workers=NUM_WORKERS)


In [7]:
from src.train import pretrain_ssl

print(f"\n>>> Full Pretraining: SimCLR ({PRETRAIN_EPOCHS} epochs)")
simclr_model = SimCLR(encoder_dim=512, projection_dim=128).to(DEVICE)
simclr_model = pretrain_ssl(simclr_model, ssl_loader, epochs=PRETRAIN_EPOCHS, lr=0.3, log_dir='runs/ssl_pretraining_simclr')

print(">>> Evaluating SimCLR with ROC-AUC...")
simclr_auc = offline_linear_probe(
    simclr_model.encoder,
    train_eval_loader,
    test_eval_loader,
    num_classes=NUM_CLASSES,
    epochs=EVAL_EPOCHS,
    device=DEVICE,
    log_dir='runs/pneumonia_simclr_linear_probe'  # Add TensorBoard log dir
)


>>> Full Pretraining: SimCLR (50 epochs)
Epoch 1/50 | Loss: 5.7585 | LR: 0.000000
Epoch 2/50 | Loss: 5.4405 | LR: 0.030000
Epoch 3/50 | Loss: 5.3694 | LR: 0.060000
Epoch 4/50 | Loss: 5.2053 | LR: 0.090000
Epoch 5/50 | Loss: 5.0074 | LR: 0.120000
Epoch 6/50 | Loss: 4.8748 | LR: 0.150000
Epoch 7/50 | Loss: 4.7956 | LR: 0.180000
Epoch 8/50 | Loss: 4.7233 | LR: 0.210000
Epoch 9/50 | Loss: 4.6904 | LR: 0.240000
Epoch 10/50 | Loss: 4.6468 | LR: 0.270000
Epoch 11/50 | Loss: 4.6100 | LR: 0.300000
Epoch 12/50 | Loss: 4.6502 | LR: 0.299538
Epoch 13/50 | Loss: 4.6307 | LR: 0.298153
Epoch 14/50 | Loss: 4.5490 | LR: 0.295855
Epoch 15/50 | Loss: 4.5145 | LR: 0.292658
Epoch 16/50 | Loss: 4.5058 | LR: 0.288582
Epoch 17/50 | Loss: 4.4825 | LR: 0.283651
Epoch 18/50 | Loss: 4.4746 | LR: 0.277896
Epoch 19/50 | Loss: 4.4693 | LR: 0.271353
Epoch 20/50 | Loss: 4.4474 | LR: 0.264061
Epoch 21/50 | Loss: 4.4397 | LR: 0.256066
Epoch 22/50 | Loss: 4.4299 | LR: 0.247417
Epoch 23/50 | Loss: 4.4252 | LR: 0.238168
E

In [8]:
print(f"\n>>> Full Pretraining: VICReg ({PRETRAIN_EPOCHS} epochs)")
vicreg_model = VICReg(encoder_dim=512, projection_dim=2048).to(DEVICE)
vicreg_model = pretrain_ssl(vicreg_model, ssl_loader, epochs=PRETRAIN_EPOCHS, lr=0.3, log_dir='runs/ssl_pretraining_vicreg')

print(">>> Evaluating VICReg with ROC-AUC...")
vicreg_auc = offline_linear_probe(
    vicreg_model.encoder,
    train_eval_loader,
    test_eval_loader,
    num_classes=NUM_CLASSES,
    epochs=EVAL_EPOCHS,
    device=DEVICE,
    log_dir='runs/pneumonia_vicreg_linear_probe'
)



>>> Full Pretraining: VICReg (50 epochs)
Epoch 1/50 | Loss: 38.7610 | LR: 0.000000
Epoch 2/50 | Loss: 38.0739 | LR: 0.030000
Epoch 3/50 | Loss: 37.9494 | LR: 0.060000
Epoch 4/50 | Loss: 37.6112 | LR: 0.090000
Epoch 5/50 | Loss: 37.1997 | LR: 0.120000
Epoch 6/50 | Loss: 36.4404 | LR: 0.150000
Epoch 7/50 | Loss: 35.7449 | LR: 0.180000
Epoch 8/50 | Loss: 35.0138 | LR: 0.210000
Epoch 9/50 | Loss: 34.2877 | LR: 0.240000
Epoch 10/50 | Loss: 34.2124 | LR: 0.270000
Epoch 11/50 | Loss: 33.2634 | LR: 0.300000
Epoch 12/50 | Loss: 32.4367 | LR: 0.299538
Epoch 13/50 | Loss: 32.1437 | LR: 0.298153
Epoch 14/50 | Loss: 31.6620 | LR: 0.295855
Epoch 15/50 | Loss: 31.2205 | LR: 0.292658
Epoch 16/50 | Loss: 30.7354 | LR: 0.288582
Epoch 17/50 | Loss: 30.2652 | LR: 0.283651
Epoch 18/50 | Loss: 29.9431 | LR: 0.277896
Epoch 19/50 | Loss: 29.6977 | LR: 0.271353
Epoch 20/50 | Loss: 29.5087 | LR: 0.264061
Epoch 21/50 | Loss: 29.1134 | LR: 0.256066
Epoch 22/50 | Loss: 28.8221 | LR: 0.247417
Epoch 23/50 | Loss: 2

In [9]:
print("\n=== Final Results (ROC-AUC) ===")
print(f"SimCLR ROC-AUC: {simclr_auc:.2f}%")
print(f"VICReg ROC-AUC: {vicreg_auc:.2f}%")


=== Final Results (ROC-AUC) ===
SimCLR ROC-AUC: 92.18%
VICReg ROC-AUC: 94.63%


## Задание 4 (1 балл)

Попробуем начать предобучение не с рандомной инициализации, а с весов, полученных предобучением на естественных картинках. Предлагается два варианта на выбор (надо выбрать один):
* веса из библиотеки `torchvision`, которые были получены supervised обучением,
* веса из соответствующих чекпоинтов [solo-learn](https://github.com/vturrisi/solo-learn/tree/main), которые были получены self-supervised обучением на Imagenet-100 (100-классовая подвыборка ImageNet'а).

Для этого при создании энкодера в `torchvision.models.resnet` можно использовать параметр `weights` у `resnet18()`. 
После инициализации с предобученных весов, проведите такой же цикл предобучения из предыдущего пункта, и продемонстрируйте разницу в финальном качестве. Помогает или вредит старт с supervised imagenet'овских весов?

In [10]:
imagenet_simclr = SimCLR(encoder_dim=512, projection_dim=128, pretrained='imagenet').to(DEVICE)
imagenet_simclr = pretrain_ssl(imagenet_simclr, ssl_loader, epochs=PRETRAIN_EPOCHS, lr=0.3)

print(">>> Evaluating ImageNet-Init SimCLR with ROC-AUC...")
imagenet_auc = offline_linear_probe(
    imagenet_simclr.encoder,
    train_eval_loader,
    test_eval_loader,
    num_classes=NUM_CLASSES,
    epochs=EVAL_EPOCHS,
    device=DEVICE,
    log_dir='runs/pneumonia_imagenet_linear_probe'
)

print("\n=== Initialization Comparison (ROC-AUC) ===")
print(f"Random Init SimCLR: {simclr_auc:.2f}%")
print(f"ImageNet Init SimCLR: {imagenet_auc:.2f}%")

Epoch 1/50 | Loss: 5.6252 | LR: 0.000000
Epoch 2/50 | Loss: 5.0684 | LR: 0.030000
Epoch 3/50 | Loss: 5.0060 | LR: 0.060000
Epoch 4/50 | Loss: 4.8898 | LR: 0.090000
Epoch 5/50 | Loss: 4.7969 | LR: 0.120000
Epoch 6/50 | Loss: 4.8333 | LR: 0.150000
Epoch 7/50 | Loss: 4.7022 | LR: 0.180000
Epoch 8/50 | Loss: 4.5875 | LR: 0.210000
Epoch 9/50 | Loss: 4.5410 | LR: 0.240000
Epoch 10/50 | Loss: 4.5260 | LR: 0.270000
Epoch 11/50 | Loss: 4.4930 | LR: 0.300000
Epoch 12/50 | Loss: 4.4577 | LR: 0.299538
Epoch 13/50 | Loss: 4.4315 | LR: 0.298153
Epoch 14/50 | Loss: 4.4155 | LR: 0.295855
Epoch 15/50 | Loss: 4.4030 | LR: 0.292658
Epoch 16/50 | Loss: 4.3939 | LR: 0.288582
Epoch 17/50 | Loss: 4.3865 | LR: 0.283651
Epoch 18/50 | Loss: 4.3795 | LR: 0.277896
Epoch 19/50 | Loss: 4.3709 | LR: 0.271353
Epoch 20/50 | Loss: 4.3671 | LR: 0.264061
Epoch 21/50 | Loss: 4.3636 | LR: 0.256066
Epoch 22/50 | Loss: 4.3568 | LR: 0.247417
Epoch 23/50 | Loss: 4.3556 | LR: 0.238168
Epoch 24/50 | Loss: 4.3502 | LR: 0.228375
E